In [1]:
import cv2
import numpy as np
import sys

In [20]:
file_name = '../fig/my_brain.jpeg'

img = cv2.imread(file_name)

if img is None:
    print('failed')
    sys.exit()
    
model = './googlenet/bvlc_googlenet.caffemodel'
config = './googlenet/deploy.prototxt'


net = cv2.dnn.readNet(model, config)

className = []
with open('./googlenet/classification_classes_ILSVRC2012.txt', 'rt') as f:
    classNames = f.read().rstrip('\n').split('\n')
    
blob = cv2.dnn.blobFromImage(img, 1, (224, 224), (100, 117, 123), False)
net.setInput(blob)
prob = net.forward()

out = prob.flatten()
classId = np.argmax(out)

confidence = out[classId]
class_text = classNames[classId]


text = f'{class_text} ({confidence*100:.2f}%)'
print(text)

cv2.putText(img, text, (10, 30), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

cv2.imshow('img', img)

cv2.waitKey()
cv2.destroyAllWindows()

Petri dish (36.31%)


In [21]:
import glob

In [39]:
img_files = glob.glob('../fig/test/*.*')

idx = 0

model = './googlenet/bvlc_googlenet.caffemodel'
config = './googlenet/deploy.prototxt'
net = cv2.dnn.readNet(model, config)
className = []
with open('./googlenet/classification_classes_ILSVRC2012.txt', 'rt') as f:
    classNames = f.read().rstrip('\n').split('\n')

while True:
    img = cv2.imread(img_files[idx])
#     cv2.resize()
    if img is None:
        print('failed')
        break
    
    blob = cv2.dnn.blobFromImage(img, 1, (224, 224), (100, 117, 123), False)
    net.setInput(blob)
    prob = net.forward()

    out = prob.flatten()
    classId = np.argmax(out)

    confidence = out[classId]
    class_text = classNames[classId]


    text1 = f'{class_text}'
    
    text2 = f'({confidence*100:.2f}%)'

    cv2.putText(img, text1, (10, 30), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
    cv2.putText(img, text2, (10, 80), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
    
    cv2.imshow('img', img)

    
    if cv2.waitKey(1000) == 27:
        break
    
    idx += 1
    if idx >= len(img_files):
        idx = 0

cv2.destroyAllWindows()

In [44]:
img = cv2.imread('./googlenet_tf/fig/restmb_allidxmake.jfif')

model = './googlenet_tf/opencv_face_detector_uint8.pb'
config = './googlenet_tf/opencv_face_detector.pbtxt'

face_net = cv2.dnn.readNet(model, config)

# blobFromImage(image[, scalefactor[, size[, mean[, swapRB[, crop[, ddepth]]]]]]) -> retva
blob = cv2.dnn.blobFromImage(img, 1, (300, 300), (104, 117, 123), swapRB = False)
face_net.setInput(blob)
out = face_net.forward()

detect = out[0, 0, :, :]

h, w = img.shape[:2]

for i in range(detect.shape[0]):
    confidence = detect[i, 2]
    
    if confidence > 0.5:
        x1 = int(detect[i, 3] * w)
        y1 = int(detect[i, 4] * h)
        x2 = int(detect[i, 5] * w)
        y2 = int(detect[i, 6] * h)
        
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 2)
        
        text = 'Face: {:.2f}%'.format(confidence*100)
        
        cv2.putText(img, text, (x1, y1-2), cv2.FONT_HERSHEY_COMPLEX,
                    1, (0, 0, 255), 1, cv2.LINE_AA)

cv2.imshow('img', img)

cv2.waitKey()
cv2.destroyAllWindows()

In [8]:
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print('failed')
    sys.exit()
    
    
model = './googlenet_tf/opencv_face_detector_uint8.pb'
config = './googlenet_tf/opencv_face_detector.pbtxt'

face_net = cv2.dnn.readNet(model, config)
    
while True:
    ret, frame = cap.read()
    
    if not ret:
        print('failed')
        break
    
    blob = cv2.dnn.blobFromImage(frame, 1, (300, 300), (104, 117, 123), swapRB = False)
    face_net.setInput(blob)
    out = face_net.forward()

    detect = out[0, 0, :, :]

    h, w = frame.shape[:2]

    for i in range(detect.shape[0]):
        confidence = detect[i, 2]
    
        if confidence > 0.2:
            x1 = int(detect[i, 3] * w)
            y1 = int(detect[i, 4] * h)
            x2 = int(detect[i, 5] * w)
            y2 = int(detect[i, 6] * h)
        
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
        
            text = 'Face: {:.2f}%'.format(confidence*100)
        
            cv2.putText(frame, text, (x1, y1-2), cv2.FONT_HERSHEY_COMPLEX,
                    1, (0, 0, 255), 1, cv2.LINE_AA)
        
#     edge =  cv2.Canny(frame, 70, 80)
    cv2.imshow('img', frame)
    
    if cv2.waitKey(1) == 27:
        break
    
cap.release()
cv2.destroyAllWindows()